In [ ]:
import pandas as pd
import glob
import os

%matplotlib inline
from collections import defaultdict, Counter
import glob
import os
import json

from IPython.core.display import HTML, Image
from matplotlib_venn import venn3
import pandas as pd
import pybedtools
import pysam
import gspread
from gscripts.general import parsers
from gscripts.general import dataviz
%load_ext autoreload
import numpy as np
%autoreload 2
reload(parsers)
reload(pybedtools)
import seaborn as sns
import matplotlib
from oauth2client.client import SignedJwtAssertionCredentials

/home/gpratt/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [ ]:
encode_qc_v12 = parsers.clipseq_metrics("/projects/ps-yeolab2/encode/analysis/encode_v12/", iclip=True)
encode_qc_v13 = parsers.clipseq_metrics("/projects/ps-yeolab/encode/analysis/encode_v13/", iclip=True)


In [ ]:
rep_files = glob.glob("/projects/ps-yeolab/encode/analysis/encode_v13/*.rmRep.metrics") + glob.glob("/projects/ps-yeolab2/encode/analysis/encode_v12/*.rmRep.metrics")
df = pd.concat({".".join(os.path.basename(fn).split(".")[:2]): pd.read_table(fn, header=None, names=["element", "count"], sep=" ", index_col=0) for fn in rep_files})
df = df.unstack().fillna(0)

encode_qc = pd.concat([encode_qc_v12, encode_qc_v13])
encode_qc_reads = encode_qc['Input Reads']
df = df.join(encode_qc_reads)


In [ ]:
df_normalized = df.div(df['Input Reads'], axis=0) * 1000000
df_normalized = df_normalized.drop("Input Reads", axis=1)
df_normalized = df_normalized.groupby(level=0).first()

In [ ]:
for x in df_normalized.index:
    print x

In [ ]:
rbp = "345_CLIP_GAGATTCC-ATAGAGGC_L002_R1.C01_345_02_PCBP2"
rep_zscores = ((df_normalized - df_normalized.mean()) / df_normalized.std()).T
rep_zscores.sort(rbp, ascending=False)[rbp]

In [ ]:
df.ix[rbp].sort_values(ascending=False)

In [ ]:
rep_zscores = (df_normalized.sub(df_normalized.mean(axis=1), axis=0).div(df_normalized.std(axis=1), axis=0)).T
rep_zscores.sort(rbp, ascending=False)[rbp]